In [1]:
import os

In [ ]:
%pwd

In [3]:
os.chdir("../")

In [ ]:
%pwd

----------------------------------- entity ------------------------------------------

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

----------------------------------- configuration manager ------------------------------------------

src/mlProject/config/configuration.py

In [6]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    # Definir el método constructor que recibe las rutas de los archivos de configuración, parámetros y esquema
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        # Leer los archivos YAML y asignarlos a los atributos de la instancia
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        # Crear el directorio raíz para los artefactos del proyecto
        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

----------------------------------- components ------------------------------------------

* ----------------------------------- data_transformation.py ------------------------------------------

In [12]:
import os
import sys
from mlProject import logger
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
import dill
import pickle


In [15]:

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler


In [103]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config


    def get_data_transformer_object(self, numerical_columns, categorical_columns):
        '''
            This function is responsible for data transformation
        '''
        try:
            # Define Pipeline
            # for numerical columns
            num_pipeline = Pipeline(
                steps = [
                    ("imputer", SimpleImputer(strategy="median")),
                    ("scaler", StandardScaler())
                ]
            )
            # for categorical columns
            cat_pipeline = Pipeline(
                steps = [
                    ("imputer", SimpleImputer(strategy="most_frequent")),
                    ("one_hot_encoder", OneHotEncoder(drop="if_binary")),
                    ("scaler", StandardScaler(with_mean=False))
                ]
            )

            # logging info
            logger.info(f"Numerical columns: {numerical_columns}")
            logger.info(f"Categorical columns: {categorical_columns}")
            
            # Combination of pipelines
            preprocessor=ColumnTransformer(
                [
                    ("num_pipeline",num_pipeline,numerical_columns),
                    ("cat_pipelines",cat_pipeline,categorical_columns)
                ]
            )

            

            return preprocessor

        except Exception as e:
            raise e


    def initiate_data_transformation(self):
        try:
            data_df = pd.read_csv(self.config.data_path)
            logger.info("Read dataset completed")

            # ----------------------------------------------------------------
            # define categorical, numerical columns and target column
            numerical_columns = ["area", "bedrooms","bathrooms","stories","parking"]
            categorical_columns = ["mainroad","guestroom","basement", "hotwaterheating","airconditioning","prefarea","furnishingstatus"]
            target_column_name = "price"

            # ----------------------------------------------------------------
            logger.info("Creating preprocessing object")
            preprocessing_obj = self.get_data_transformer_object(numerical_columns, categorical_columns)

            # ----------------------------------------------------------------
            # define variable X and y for training data
            input_feature_df=data_df.drop(columns=[target_column_name],axis=1)
            target_feature_df=data_df[target_column_name]

            # ----------------------------------------------------------------
            logger.info(f"Applying preprocessing object on training and testing dataframe")
            input_feature_arr = preprocessing_obj.fit_transform(input_feature_df)

            # ----------------------------------------------------------------

            # concatenate preprocessed input features with target feature
            data_arr = np.c_[input_feature_arr, np.array(target_feature_df)]
            # define the column names as a list
            column_names = list(preprocessing_obj.get_feature_names_out()) + [target_column_name]
            # removing prefixes of columns names
            column_names = [name.replace("cat_pipelines__", "").replace("num_pipeline__", "") for name in column_names]
            # convert the array to a dataframe
            data_df = pd.DataFrame(data_arr, columns=column_names)


            # ----------------------------------------------------------------
            # Split the data into training and test sets. (0.75, 0.25) split.
            train, test = train_test_split(data_df)#data_arr)
            logger.info("Splited data into training and test sets")

            logger.info(train.shape)
            logger.info(test.shape)

            # ----------------------------------------------------------------
            logger.info(f"saving the pre-processing data")
            train.to_csv(os.path.join(self.config.root_dir, "train.csv"),index = False)
            test.to_csv(os.path.join(self.config.root_dir, "test.csv"),index = False)

            
            # ----------------------------------------------------------------
            # save object
            ruta = self.config.root_dir + '/preprocessing_obj.pkl'
            with open(ruta, 'wb') as f:
                pickle.dump(preprocessing_obj, f)
                
            logger.info(f"Saved preprocessing object")

            # ----------------------------------------------------------------
            return (
                preprocessing_obj,)

        except Exception as e:
            raise e
        

----------------------------------- pipeline ------------------------------------------

* ----------------------------------- stage_03_data_transformation_pipeline ------------------------------------------

In [104]:
"""
Realizar la transformacion de los datos
"""
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    pre_process_obj = data_transformation.initiate_data_transformation()
except Exception as e:
    raise e

[2023-08-19 16:33:36,693: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-19 16:33:36,701: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-19 16:33:36,711: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-08-19 16:33:36,716: INFO: common: created directory at: artifacts]
[2023-08-19 16:33:36,721: INFO: common: created directory at: artifacts/data_transformation]
[2023-08-19 16:33:36,731: INFO: 3059530493: Read dataset completed]
[2023-08-19 16:33:36,736: INFO: 3059530493: Creating preprocessing object]
[2023-08-19 16:33:36,738: INFO: 3059530493: Numerical columns: ['area', 'bedrooms', 'bathrooms', 'stories', 'parking']]
[2023-08-19 16:33:36,742: INFO: 3059530493: Categorical columns: ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea', 'furnishingstatus']]
[2023-08-19 16:33:36,749: INFO: 3059530493: Applying preprocessing object on training and testing dataframe]
[2023-08-19 16:33:36,802: INF

[2023-08-19 16:33:36,863: INFO: 3059530493: Saved preprocessing object]
